## 대안 데이터 수집

In [17]:
import requests
import xml.etree.ElementTree as ElementTree
import pandas as pd

In [ ]:
url = 'http://apis.data.go.kr/9710000/BillInfoService2/getBillAdditionalInfo'
params = {
    'serviceKey': 'UJY+e286zOQsAHMHd/5cggpYFaFqG5mWawJKgrubJeKRBqVp0VUsyeHIgw/VGPQjWRSp6yaR/sUhXlhpKyv1cg==',
    'bill_id': 'PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5'
}

# 데이터 수집 시작
all_data = []
pageNo = 1
processing_count = 0
max_retry=3

while True:
    params.update({'pageNo': str(pageNo)})
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        print('응답 성공')
        print(response.content)
        try:
            root = ElementTree.fromstring(response.content)
            print(root)
            items = root.find('body').find('items')
            print(items)
            
            if items is None or len(items) == 0:
                # print("No more data available.")
                break
            
            data = [{child.tag: child.text for child in item} for item in items]
            all_data.extend(data)
            
            # print(f"Page {pageNo} processed. {len(data)} items added. total: {len(all_data)}")
            processing_count += 1
            
        except ElementTree.ParseError:
            print(f"XML Parsing Error: {response.text}")
            max_retry -= 1
        except Exception as e:
            print(f"Unexpected Error: {e}")
            max_retry -= 1
    else:
        print(f"Error Code: {response.status_code} (Page {pageNo})")
        max_retry -= 1
    
    if max_retry <= 0:
        print("Maximum retry reached. Exiting...")
        break
    
    pageNo += 1

# 데이터프레임 생성
df_billsAlternatives = pd.DataFrame(all_data)
df_billsAlternatives

print('실행 완료')

응답 성공
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><requestMsgID></requestMsgID><responseTime>2024-12-19 20:04:22.422</responseTime><responseMsgID></responseMsgID><successYN></successYN><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><billGbnCd/><bpmOthers/><commMemo/><exhaust><item><billLink>http://likms.assembly.go.kr/bill/jsp/BillDetail.jsp?bill_id=PRC_A1M2H1A1P1M6C1F5S1B5X5Z7I4B9V4</billLink><billName>\xeb\x8f\x84\xeb\xa1\x9c\xea\xb5\x90\xed\x86\xb5\xeb\xb2\x95 \xec\x9d\xbc\xeb\xb6\x80\xea\xb0\x9c\xec\xa0\x95\xeb\xb2\x95\xeb\xa5\xa0\xec\x95\x88(\xeb\xb0\xb1\xec\x9e\xac\xed\x98\x84\xec\x9d\x98\xec\x9b\x90 \xeb\x93\xb1 12\xec\x9d\xb8\xeb\xb0\x9c\xec\x9d\x98)</billName></item><item><billLink>http://likms.assembly.go.kr/bill/jsp/BillDetail.jsp?bill_id=PRC_W1O4W0K1B2N0P1C0Z5T1W4F9A7N3Z7</billLink><billName>\xeb\x8f\x84\xeb\xa1\x9c\xea\xb5\x90\xed\x86\xb5\xeb\xb2\x95 \xec\x9d\xbc\xeb\xb6\x80\xea\xb0\x9c\xec\xa0\x95\xeb\xb2\x95

In [1]:
import requests
import xml.etree.ElementTree as ElementTree
import pandas as pd

# API 요청 및 데이터 파싱 함수
def fetch_law_data(bill_id):
    url = 'http://apis.data.go.kr/9710000/BillInfoService2/getBillAdditionalInfo'
    params = {
        'serviceKey': 'UJY+e286zOQsAHMHd/5cggpYFaFqG5mWawJKgrubJeKRBqVp0VUsyeHIgw/VGPQjWRSp6yaR/sUhXlhpKyv1cg==',
        'bill_id': bill_id
    }
    
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            root = ElementTree.fromstring(response.content)
            items = root.find('.//exhaust')
            
            if items is None or len(items.findall('item')) == 0:
                return []
            
            # 법률안 데이터 추출
            law_data = []
            for item in items.findall('item'):
                bill_link = item.find('billLink').text
                law_bill_id = bill_link.split('bill_id=')[-1]
                bill_name = item.find('billName').text.encode('utf-8').decode('utf-8')  # 한글 디코딩
                law_data.append({'billId': law_bill_id, 'billName': bill_name})
            return law_data
        else:
            print(f"Error for bill_id={bill_id}, status_code={response.status_code}")
            return []
    except Exception as e:
        print(f"Error for bill_id={bill_id}: {e}")
        return []

# 대안별로 법률안을 수집하여 데이터프레임 생성
def collect_alternative_data(df_alternatives):
    all_data = []

    for _, row in df_alternatives.iterrows():
        alt_id = row['billId']  # 대안의 bill_id
        law_data = fetch_law_data(alt_id)  # 해당 대안의 법률안 데이터 호출
        
        for law in law_data:
            all_data.append({'AltId': alt_id, 'billName': law['billName'], 'billId': law['billId']})
    
    # 최종 데이터프레임 생성
    final_df = pd.DataFrame(all_data)
    return final_df

# 테스트 데이터프레임 (대안 데이터)
df_alternatives = pd.DataFrame({
    'billId': ['PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5', 'PRC_P2X4L1A1M2D7V1C9P3U5H5J5E4Z2Q5'],
    'billName': ['대안1', '대안2']
})

# 데이터 수집 및 결과 출력
final_df = collect_alternative_data(df_alternatives)

# 결과 저장
# final_df.to_csv('bills_alternatives_data.csv', index=False, encoding='utf-8-sig')
final_df

,AltId,billName,billId
0,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(백재현의원 등 12인발의),PRC_A1M2H1A1P1M6C1F5S1B5X5Z7I4B9V4
1,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(박남춘의원 등 13인발의),PRC_W1O4W0K1B2N0P1C0Z5T1W4F9A7N3Z7
2,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(이노근의원 등 11인발의),PRC_J1M4J0P3M0A5C1J1X4P5V1H2J9J1J4
3,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(최민희의원 등 10인발의),PRC_P1J4D0G4Q1O4Q1L6K1U0D0P6R8U3Z0
4,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(진선미의원 등 10인발의),PRC_Q1C4F1B1Y0O3X2H0Y2F6Q2T7L5Y0E3
5,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(이철우의원 등 10인발의),PRC_J1D4H1D1W1S4X1L7E4C5P3M9W7Y7F4
6,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(김민기의원 등 10인발의),PRC_A1S4V0W7N1D4X1Z1E2N2X1Z4C9K9Q4
7,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(박남춘의원 등 10인발의),PRC_R1Z4E1E1U2V1J1R0L2V8L2K1A6L5F8
8,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(정부발의),ARC_Z1Q4T1B2O3R1E1Q4D2Y7O0W1B9M1Z8
9,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,도로교통법 일부개정법률안(김민기의원 등 11인발의),PRC_B1Z5F0L1L2M0E1J4A4Z6W4Z9U1Q3D8


In [34]:
import requests
import xml.etree.ElementTree as ElementTree
import pandas as pd

url = 'http://apis.data.go.kr/9710000/BillInfoService2/getBillAdditionalInfo'
params = {
    'serviceKey': 'UJY+e286zOQsAHMHd/5cggpYFaFqG5mWawJKgrubJeKRBqVp0VUsyeHIgw/VGPQjWRSp6yaR/sUhXlhpKyv1cg==',
    'bill_id': 'PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5'
}

# 데이터 수집 및 파싱
all_data = []

response = requests.get(url, params=params)

if response.status_code == 200:
    print('응답 성공')
    try:
        # XML 데이터를 파싱
        root = ElementTree.fromstring(response.content)
        items = root.find('.//exhaust')  # <exhaust> 아래의 데이터를 찾음

        if items is None or len(items.findall('item')) == 0:
            print("데이터가 없습니다.")
        else:
            # 아이템 데이터 추출
            for item in items.findall('item'):
                bill_link = item.find('billLink').text
                bill_name = item.find('billName').text.encode('utf-8').decode('utf-8')  # 한글 디코딩
                all_data.append({'billLink': bill_link, 'billName': bill_name})

    except ElementTree.ParseError:
        print(f"XML Parsing Error: {response.content}")
    except Exception as e:
        print(f"Unexpected Error: {e}")
else:
    print(f"Error Code: {response.status_code}")

# 데이터프레임 생성
df_billsAlternatives = pd.DataFrame(all_data)

# bill_id 컬럼 추가
if not df_billsAlternatives.empty:
    df_billsAlternatives['bill_id'] = df_billsAlternatives['billLink'].str.extract(r'bill_id=([^&]+)')

# 결과 출력
# print(df_billsAlternatives)

# 데이터 저장
# df_billsAlternatives.to_csv('bills_alternatives_with_ids.csv', index=False, encoding='utf-8-sig')
print('실행 완료')

df_billsAlternatives


응답 성공
실행 완료


,billLink,billName,bill_id
0,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(백재현의원 등 12인발의),PRC_A1M2H1A1P1M6C1F5S1B5X5Z7I4B9V4
1,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(박남춘의원 등 13인발의),PRC_W1O4W0K1B2N0P1C0Z5T1W4F9A7N3Z7
2,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(이노근의원 등 11인발의),PRC_J1M4J0P3M0A5C1J1X4P5V1H2J9J1J4
3,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(최민희의원 등 10인발의),PRC_P1J4D0G4Q1O4Q1L6K1U0D0P6R8U3Z0
4,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(진선미의원 등 10인발의),PRC_Q1C4F1B1Y0O3X2H0Y2F6Q2T7L5Y0E3
5,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(이철우의원 등 10인발의),PRC_J1D4H1D1W1S4X1L7E4C5P3M9W7Y7F4
6,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(김민기의원 등 10인발의),PRC_A1S4V0W7N1D4X1Z1E2N2X1Z4C9K9Q4
7,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(박남춘의원 등 10인발의),PRC_R1Z4E1E1U2V1J1R0L2V8L2K1A6L5F8
8,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(정부발의),ARC_Z1Q4T1B2O3R1E1Q4D2Y7O0W1B9M1Z8
9,http://likms.assembly.go.kr/bill/jsp/BillDetai...,도로교통법 일부개정법률안(김민기의원 등 11인발의),PRC_B1Z5F0L1L2M0E1J4A4Z6W4Z9U1Q3D8


In [35]:
df_billsAlternatives['billLink'][0]

'http://likms.assembly.go.kr/bill/jsp/BillDetail.jsp?bill_id=PRC_A1M2H1A1P1M6C1F5S1B5X5Z7I4B9V4'

In [6]:
# 테스트 코드
if __name__ == "__main__":
    # 테스트용 데이터프레임 생성
    # billId 값은 실제 API 호출 시 유효한 값으로 대체해야 합니다.
    df_test_bills = pd.DataFrame({
        'billId': ['PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5',  # 테스트용 billId
                   'PRC_P2X4L1A1M2D7V1C9P3U5H5J5E4Z2Q5'],  # 추가 billId
        'billName': ['테스트 법안1', '테스트 법안2']  # 법안 이름
    })

    # 함수 호출
    print("대안 법안 데이터 수집 중...")
    df_alternatives = fetch_bills_alternatives(df_test_bills)
    
    # 결과 출력
    print("\n수집된 대안 데이터:")
    print(df_alternatives)

    # 결과를 CSV로 저장 (옵션)
    # df_alternatives.to_csv("test_alternatives_output.csv", index=False, encoding='utf-8-sig')

대안 법안 데이터 수집 중...

수집된 대안 데이터:
                                billId                           altBillId
0   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_A1M2H1A1P1M6C1F5S1B5X5Z7I4B9V4
1   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_W1O4W0K1B2N0P1C0Z5T1W4F9A7N3Z7
2   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_J1M4J0P3M0A5C1J1X4P5V1H2J9J1J4
3   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_P1J4D0G4Q1O4Q1L6K1U0D0P6R8U3Z0
4   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_Q1C4F1B1Y0O3X2H0Y2F6Q2T7L5Y0E3
5   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_J1D4H1D1W1S4X1L7E4C5P3M9W7Y7F4
6   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_A1S4V0W7N1D4X1Z1E2N2X1Z4C9K9Q4
7   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_R1Z4E1E1U2V1J1R0L2V8L2K1A6L5F8
8   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  ARC_Z1Q4T1B2O3R1E1Q4D2Y7O0W1B9M1Z8
9   PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_B1Z5F0L1L2M0E1J4A4Z6W4Z9U1Q3D8
10  PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_H1I4B1K2U1Z2C1P8E0C2T2Q2A4O1Z2
11  PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5  PRC_D1S5R0H2K2O4E1T3E5W5Y4L4U

In [7]:
df_alternatives

,billId,altBillId
0,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_A1M2H1A1P1M6C1F5S1B5X5Z7I4B9V4
1,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_W1O4W0K1B2N0P1C0Z5T1W4F9A7N3Z7
2,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_J1M4J0P3M0A5C1J1X4P5V1H2J9J1J4
3,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_P1J4D0G4Q1O4Q1L6K1U0D0P6R8U3Z0
4,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_Q1C4F1B1Y0O3X2H0Y2F6Q2T7L5Y0E3
5,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_J1D4H1D1W1S4X1L7E4C5P3M9W7Y7F4
6,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_A1S4V0W7N1D4X1Z1E2N2X1Z4C9K9Q4
7,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_R1Z4E1E1U2V1J1R0L2V8L2K1A6L5F8
8,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,ARC_Z1Q4T1B2O3R1E1Q4D2Y7O0W1B9M1Z8
9,PRC_G1U5T0V4E2J8K1Q3Q3G7L5N0Z1D5G5,PRC_B1Z5F0L1L2M0E1J4A4Z6W4Z9U1Q3D8


In [ ]:


api_key = os.environ.get("APIKEY_billsContent")
url = 'http://apis.data.go.kr/9710000/BillInfoService2/getBillInfoList'
params = {
    'serviceKey': api_key,
    'numOfRows': '100',
    'start_ord': age,
    'end_ord': age,
    'start_propose_date': start_date,
    'end_propose_date': end_date
}

# 수집하는 날짜 범위 출력
print(f"[{start_date} ~ {end_date} 의안주요내용 데이터 수집 시작]")

# 데이터 수집 시작
all_data = []
pageNo = 1
processing_count = 0
max_retry=3

start_time = time.time()

while True:
    params.update({'pageNo': str(pageNo)})
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        try:
            root = ElementTree.fromstring(response.content)
            items = root.find('body').find('items')
            
            if items is None or len(items) == 0:
                # print("No more data available.")
                break
            
            data = [{child.tag: child.text for child in item} for item in items]
            all_data.extend(data)
            
            # print(f"Page {pageNo} processed. {len(data)} items added. total: {len(all_data)}")
            processing_count += 1
            
        except ElementTree.ParseError:
            print(f"XML Parsing Error: {response.text}")
            max_retry -= 1
        except Exception as e:
            print(f"Unexpected Error: {e}")
            max_retry -= 1
    else:
        print(f"Error Code: {response.status_code} (Page {pageNo})")
        max_retry -= 1
    
    if max_retry <= 0:
        print("Maximum retry reached. Exiting...")
        break
    
    pageNo += 1

# 데이터프레임 생성
df_billsContent = pd.DataFrame(all_data)

end_time = time.time()
total_time = end_time - start_time
print(f"[모든 파일 다운로드 완료! 전체 소요 시간: {total_time:.2f}초]")
print(f"[{len(df_billsContent)} 개의 법안 수집됨.]")

# 수집한 데이터가 없으면 종료
if len(df_billsContent) == 0:
    return None

# 유지할 컬럼 목록
columns_to_keep = [
    'proposeDt', # 발의일자
    'billNo', # 법안번호
    # 'billName', # 법안명
    'summary', # 주요내용
    'procStageCd', # 현재 처리 단계
    # 'generalResult' # 처리 결과
    'proposerKind'
]

# 지정된 컬럼만 유지하고 나머지 제거
df_billsContent = df_billsContent[columns_to_keep]

# 'summary' 컬럼에 결측치가 있는 행 제거
df_billsContent = df_billsContent.dropna(subset=['summary'])

# 인덱스 재설정
df_billsContent.reset_index(drop=True, inplace=True)

# 컬럼 이름 변경
df_billsContent.rename(columns={
    "proposeDt": "proposeDate",
    "billNo": "billNumber",
    "summary": "summary",
    "procStageCd": "stage"
    }, inplace=True)

print(f"[결측치 처리 완료. {len(df_billsContent)} 개의 법안 수집됨.]")
print("\n발의일자별 수집한 데이터 수 :")
print(f"{df_billsContent['proposeDate'].value_counts()}")    

return df_billsContent

## 250111 작업

In [1]:
def fetch_bills_alternatives(df_bills):
    """
    입력받은 법안 데이터프레임을 기반으로 각 법안의 대안을 수집하고 반환하는 함수.
    
    Parameters:
    df_bills (pd.DataFrame): 법안 데이터프레임 (billId와 billName 열 포함)
    
    Returns:
    pd.DataFrame: 각 법안의 대안을 포함하는 데이터프레임
    """
    def fetch_alternativeBills_relation_data(bill_id):
        # API 요청 URL 및 파라미터 설정
        url = 'http://apis.data.go.kr/9710000/BillInfoService2/getBillAdditionalInfo'
        params = {
            'serviceKey': 'UJY+e286zOQsAHMHd/5cggpYFaFqG5mWawJKgrubJeKRBqVp0VUsyeHIgw/VGPQjWRSp6yaR/sUhXlhpKyv1cg==',
            'bill_id': bill_id
        }
        
        try:
            # API 요청
            response = requests.get(url, params=params)
            if response.status_code == 200:
                # 응답 데이터 파싱
                root = ElementTree.fromstring(response.content)
                items = root.find('.//exhaust')
                
                if items is None or len(items.findall('item')) == 0:
                    return []
                
                # 법률안 데이터 추출
                law_data = []
                for item in items.findall('item'):
                    bill_link = item.find('billLink').text
                    law_bill_id = bill_link.split('bill_id=')[-1]
                    bill_name = item.find('billName').text.encode('utf-8').decode('utf-8')  # 한글 디코딩
                    law_data.append({'billId': law_bill_id, 'billName': bill_name})
                return law_data
            else:
                print(f"Error for bill_id={bill_id}, status_code={response.status_code}")
                return []
        except Exception as e:
            print(f"Error for bill_id={bill_id}: {e}")
            return []
    
    # 대안 데이터프레임 초기화
    alternatives_data = []

    # 각 법안의 billId를 이용해 대안 데이터 수집
    for _, row in df_bills.iterrows():
        bill_id = row['billId']  # 법안 ID
        # bill_name = row['billName']  # 법안 이름
        
        # 대안 데이터 수집
        law_data = fetch_alternativeBills_relation_data(bill_id)
        
        # 수집된 데이터를 정리하여 리스트에 추가
        for law in law_data:
            alternatives_data.append({
                'billId': bill_id,  # 원 법안 ID
                # 'originalBillName': bill_name,  # 원 법안 이름
                'altBillId': law['billId'],  # 대안 법안 ID
                # 'altBillName': law['billName']  # 대안 법안 이름
            })
    
    # 대안 데이터를 데이터프레임으로 변환
    df_alternatives = pd.DataFrame(alternatives_data)
    return df_alternatives

In [1]:
import requests
import pandas as pd
from xml.etree import ElementTree
import time
from datetime import datetime, timedelta
from IPython.display import clear_output 
from openai import OpenAI
import json
import os
import pymysql
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import sys

In [15]:
load_dotenv()

api_key = os.environ.get("APIKEY_billsContent")
url = 'http://apis.data.go.kr/9710000/BillInfoService2/getBillInfoList'
params = {
    'serviceKey': api_key,
    'numOfRows': '100',
    'start_ord': '22',
    'end_ord': '22',
    # 'start_propose_date': start_date,
    # 'end_propose_date': end_date,
    'proposer_kind_cd': 'F02'
}

# 수집하는 날짜 범위 출력
# print(f"[{start_date} ~ {end_date} 의안주요내용 데이터 수집 시작]")

# 데이터 수집 시작
all_data = []
pageNo = 1
processing_count = 0
max_retry=3

start_time = time.time()

while True:
    params.update({'pageNo': str(pageNo)})
    response = requests.get(url, params=params)
    
    print(response.content)
    
    if response.status_code == 200:
        try:
            root = ElementTree.fromstring(response.content)
            items = root.find('body').find('items')
            
            if items is None or len(items) == 0:
                # print("No more data available.")
                break
            
            data = [{child.tag: child.text for child in item} for item in items]
            all_data.extend(data)
            
            # print(f"Page {pageNo} processed. {len(data)} items added. total: {len(all_data)}")
            processing_count += 1
            
        except ElementTree.ParseError:
            print(f"XML Parsing Error: {response.text}")
            max_retry -= 1
        except Exception as e:
            print(f"Unexpected Error: {e}")
            max_retry -= 1
    else:
        print(f"Error Code: {response.status_code} (Page {pageNo})")
        max_retry -= 1
    
    if max_retry <= 0:
        print("Maximum retry reached. Exiting...")
        break
    
    pageNo += 1

# 데이터프레임 생성
df_billsContent = pd.DataFrame(all_data)

end_time = time.time()
total_time = end_time - start_time
print(f"[모든 파일 다운로드 완료! 전체 소요 시간: {total_time:.2f}초]")
print(f"[{len(df_billsContent)} 개의 법안 수집됨.]")

# 수집한 데이터가 없으면 종료
if len(df_billsContent) == 0:
    print("수집한 데이터가 없습니다. 코드를 종료합니다.")
    # return None

# 유지할 컬럼 목록
columns_to_keep = [
    'proposeDt', # 발의일자
    'billNo', # 법안번호
    # 'billName', # 법안명
    'summary', # 주요내용
    'procStageCd', # 현재 처리 단계
    # 'generalResult' # 처리 결과
    'proposerKind'
]

# 지정된 컬럼만 유지하고 나머지 제거
df_billsContent = df_billsContent[columns_to_keep]

# 'summary' 컬럼에 결측치가 있는 행 제거
df_billsContent = df_billsContent.dropna(subset=['summary'])

# 인덱스 재설정
df_billsContent.reset_index(drop=True, inplace=True)

# 컬럼 이름 변경
df_billsContent.rename(columns={
    "proposeDt": "proposeDate",
    "billNo": "billNumber",
    "summary": "summary",
    "procStageCd": "stage"
    }, inplace=True)

print(f"[결측치 처리 완료. {len(df_billsContent)} 개의 법안 수집됨.]")
print("\n발의일자별 수집한 데이터 수 :")
print(f"{df_billsContent['proposeDate'].value_counts()}")    


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><billId>PRC_D2A5C0V1A0G9Y1S1C3J7I1T6Z3C4M9</billId><billName>\xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9 \xea\xb4\x80\xec\xa0\x80 \xec\x9d\xb4\xec\xa0\x84 \xea\xb4\x80\xeb\xa0\xa8 \xea\xb0\x90\xec\x82\xac\xec\x9b\x90\xec\x97\x90 \xeb\x8c\x80\xed\x95\x9c \xea\xb0\x90\xec\x82\xac\xec\x9a\x94\xea\xb5\xac\xec\x95\x88(\xea\xb5\xad\xed\x9a\x8c\xec\x9a\xb4\xec\x98\x81\xec\x9c\x84\xec\x9b\x90\xec\x9e\xa5)</billName><billNo>2207403</billNo><passGubn>\xea\xb3\x84\xeb\xa5\x98\xec\x9d\x98\xec\x95\x88</passGubn><procStageCd>\xeb\xb3\xb8\xed\x9a\x8c\xec\x9d\x98\xeb\xb6\x80\xec\x9d\x98\xec\x95\x88\xea\xb1\xb4</procStageCd><proposeDt>2025-01-09</proposeDt><proposerKind>\xec\x9c\x84\xec\x9b\x90\xec\x9e\xa5</proposerKind></item><item><billId>PRC_L2C4R1D2E0V2F1D1K3M2T3W3C4D9L5</billId><billName>\xea\xb8\xb0\xec\x97\x85\xeb\xb6\x80\xec\x84\xa4\xec

In [11]:
df_billsContent

,proposeDate,billNumber,summary,stage,proposerKind
0,2025-01-07,2207310,대안의 제안이유\r\n 기업의 연구개발 활동을 관리 및 지원하기 위해 1981년 ...,본회의의결,위원장
1,2025-01-07,2207307,"대안의 제안이유\r\n 최근 도시권역 인구집중, 지방소멸위기 등의 사회 환경 변화...",본회의의결,위원장
2,2025-01-07,2207306,대안의 제안이유 및 주요내용\r\n 현행법은 항공운송사업자에게 운송 불이행 및 지...,본회의의결,위원장
3,2024-12-30,2207090,"2. 대안의 제안이유\r\n 인공유방, 인공관절 등 인체에 30일 이상 삽입되는 ...",본회의의결,위원장
4,2024-12-30,2207087,대안의 제안이유\r\n 최근 기후위기에 따른 기상이변과 지속적인 산림개발로 산불ㆍ...,본회의의결,위원장
...,...,...,...,...,...
150,2024-08-27,2203262,대안의 제안이유 및 주요내용\r\n\r\n 가. 2024년 9월 20일까지인 도심...,공포,위원장
151,2024-07-31,2202444,대안의 제안이유\r\n\r\n 헌법에 보장된 ‘노동3권’은 법률이 없더라도 헌법의...,재의(부결),위원장
152,2024-06-26,2200948,대안의 제안이유\r\n 한국교육방송공사가 공적 책임을 실현하기 위해서는 각 방송...,재의(부결),위원장
153,2024-06-26,2200947,대안의 제안이유\r\n 방송문화진흥회가 최다출자자인 방송사업자(이하 “문화방송”이...,재의(부결),위원장


In [1]:
import data_operations as dataops

df_alt_test = dataops.update_bills_alternatives(fetch_mode='total')

df_alt_test

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><billId>PRC_D2A5C0V1A0G9Y1S1C3J7I1T6Z3C4M9</billId><billName>\xeb\x8c\x80\xed\x86\xb5\xeb\xa0\xb9 \xea\xb4\x80\xec\xa0\x80 \xec\x9d\xb4\xec\xa0\x84 \xea\xb4\x80\xeb\xa0\xa8 \xea\xb0\x90\xec\x82\xac\xec\x9b\x90\xec\x97\x90 \xeb\x8c\x80\xed\x95\x9c \xea\xb0\x90\xec\x82\xac\xec\x9a\x94\xea\xb5\xac\xec\x95\x88(\xea\xb5\xad\xed\x9a\x8c\xec\x9a\xb4\xec\x98\x81\xec\x9c\x84\xec\x9b\x90\xec\x9e\xa5)</billName><billNo>2207403</billNo><passGubn>\xea\xb3\x84\xeb\xa5\x98\xec\x9d\x98\xec\x95\x88</passGubn><procStageCd>\xeb\xb3\xb8\xed\x9a\x8c\xec\x9d\x98\xeb\xb6\x80\xec\x9d\x98\xec\x95\x88\xea\xb1\xb4</procStageCd><proposeDt>2025-01-09</proposeDt><proposerKind>\xec\x9c\x84\xec\x9b\x90\xec\x9e\xa5</proposerKind></item><item><billId>PRC_L2C4R1D2E0V2F1D1K3M2T3W3C4D9L5</billId><billName>\xea\xb8\xb0\xec\x97\x85\xeb\xb6\x80\xec\x84\xa4\xec

100%|██████████| 181/181 [00:33<00:00,  5.41it/s]

[22대 전체 대안-법안 관계 데이터 수집 완료됨]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   altBillId  769 non-null    object
 1   billId     769 non-null    object
dtypes: object(2)
memory usage: 12.1+ KB
None


,altBillId,billId
0,PRC_L2C4R1D2E0V2F1D1K3M2T3W3C4D9L5,PRC_B2S4N0H5P3R0O1X0V4X8S2K1R9S2K7
1,PRC_L2C4R1D2E0V2F1D1K3M2T3W3C4D9L5,PRC_Y2H4F0E6D0E3M1L0K4I7R5P3O7N3J2
2,PRC_S2M4X1X2K1S9N1M3N5U5J3U1A5C1O1,PRC_X2V4W0U6U1T0I2I0H3I1G0E1F1N8O3
3,PRC_S2M4X1X2K1S9N1M3N5U5J3U1A5C1O1,PRC_X2Y4G0E7D2L6I1F6D4E4D2B0N0N9N5
4,PRC_S2M4X1X2K1S9N1M3N5U5J3U1A5C1O1,PRC_G2H4G0E7D2L6K1I6J4I6K5J9I8E5D9
...,...,...
764,PRC_A2E4G0C6K1K8P1M1E5X0I2B5I5Z5E2,PRC_R2Y4G0U5I3R1I1W7K4M5T0T1S8Z1B3
765,PRC_A2E4G0C6K1K8P1M1E5X0I2B5I5Z5E2,PRC_R2N4F0T6X0R3T1M9F0E2L0Z9M6Z0I3
766,PRC_A2E4G0C6K1K8P1M1E5X0I2B5I5Z5E2,PRC_R2G4C0N6F0I7X1H8U2S1C3F2P2I6L6
767,PRC_A2E4G0C6K1K8P1M1E5X0I2B5I5Z5E2,PRC_I2J4Q0R6P0Q6O1P7N5V9V5U5V5T4T2


In [3]:
df_alt_test.to_csv('bills_alternatives_relation_data_22.csv')

In [4]:
df_alt_test.to_json('bills_alternatives_relation_data_22.json')